In [23]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [24]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=gemini_api_key)

llm.invoke([HumanMessage("테슬라는 한달 전에 비해 주가가 올랐나 내렸나?")])

AIMessage(content='최근 한 달 동안 테슬라 주가가 어떻게 변동했는지 알려드리겠습니다.\n\n(2024년 5월 15일)을 기준으로 한 달 전인 (2024년 4월 15일)의 테슬라 주가를 찾아보면, 대략 170달러 정도였습니다.\n\n현재(2024년 5월 15일) 테슬라 주가는 190달러 정도입니다.\n\n따라서 한 달 전에 비해 테슬라 주가는 **상승**했습니다.\n\n하지만 주가는 계속 변동하기 때문에 투자 결정을 내리기 전에는 반드시 최신 정보를 확인하시기 바랍니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--d2032e4a-c33a-4a9d-a66a-40401cbba0e6-0', usage_metadata={'input_tokens': 19, 'output_tokens': 180, 'total_tokens': 199, 'input_token_details': {'cache_read': 0}})

In [25]:
from langchain_core.tools import tool
from datetime import datetime
from zoneinfo import ZoneInfo

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    target_timezone = ZoneInfo(timezone)
    now = datetime.now(target_timezone).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time

### 파이단틱(Pydantic)

*   **데이터 유효성 검사**: 런타임에 데이터 유효성을 강제하여 오류를 조기에 발견하고 안정적인 애플리케이션을 구축할 수 있게 한다. 
*   **설정 관리**: 환경 변수, 파일 등에서 애플리케이션 설정을 로드하고 유효성을 검사하여 안전하고 구성 가능한 애플리케이션을 만들 수 있게 한다.
*   **데이터 직렬화 및 역직렬화**: Python 객체를 JSON과 같은 형식으로 쉽게 변환하고 다시 Python 객체로 역변환할 수 있게 한다.
*   **문서화**: Pydantic 모델은 OpenAPI(Swagger)와 같은 도구에서 자동으로 API 문서를 생성하는 데 사용될 수 있게 한다.

In [26]:
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")

### Yahoo Finance API 사용

In [27]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    """ 주식 종목의 가격 데이터를 조회하는 함수"""
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown() 

    return history_md

In [28]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

# 도구들을 tools 리스트에 추가
tools = [get_current_time, get_yf_stock_history]

# 에이전트 프롬프트 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# 에이전트 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 초기화
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [29]:
from langchain_core.messages import SystemMessage

# 메시지 형식 변경 (agent_executor에 맞게)
messages = [
    HumanMessage("테슬라는 한달 전에 비해 주가가 올랐나 내렸나?"),
]

# agent_executor를 사용하여 실행
response = agent_executor.invoke({"input": messages})



> Entering new AgentExecutor chain...

Invoking: `get_yf_stock_history` with `{'stock_history_input': {'period': '1mo', 'ticker': 'TSLA'}}`


| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |
|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|
| 2025-07-07 00:00:00-04:00 | 291.37 | 296.15 | 288.77 |  293.94 | 1.31178e+08 |           0 |              0 |
| 2025-07-08 00:00:00-04:00 | 297    | 304.05 | 294.35 |  297.81 | 1.03247e+08 |           0 |              0 |
| 2025-07-09 00:00:00-04:00 | 297.55 | 300.15 | 293.55 |  295.88 | 7.55868e+07 |           0 |              0 |
| 2025-07-10 00:00:00-04:00 | 300.05 | 310.48 | 300    |  309.87 | 1.04365e+08 |           0 |              0 |
| 2025-07-11 00:00:00-04:00 | 307.89 | 314.09 | 305.65 |  313.51 | 7.92364e+07 |           0 |              0 |
| 2025-07-14 00:00:00-04:00 | 317.73 | 322.6  | 312.67 |  316.9  | 7.804

In [32]:
response

{'input': [HumanMessage(content='테슬라는 한달 전에 비해 주가가 올랐나 내렸나?', additional_kwargs={}, response_metadata={})],
 'output': '한 달 전 테슬라 주가는 2025-07-07 종가 기준으로 293.94 달러였고, 현재(2025-08-06)는 319.91 달러입니다. 따라서 한 달 전에 비해 주가가 올랐습니다.'}